# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [31]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt


In [5]:
spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .getOrCreate()

In [6]:
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

# Question 1

Which page did user id "" (empty string) NOT visit?

In [20]:
# TODO: write your code to answer question 1

pages = user_log.filter(user_log.userId == '').select(col('page').alias('blank_pages')).dropDuplicates()

all_pages = user_log.select('page').dropDuplicates()

for row in set(all_pages.collect()) - set(pages.collect()):
    print(row.page)

Settings
Downgrade
NextSong
Error
Upgrade
Submit Downgrade
Save Settings
Submit Upgrade
Logout


# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [2]:
# TODO: use this space to explore the behavior of the user with an empty string
"""Users that are going to sign up, sign out, or log in"""

# Question 3

How many female users do we have in the data set?

In [22]:
# TODO: write your code to answer question 3

user_log.filter(user_log.gender == 'F').select('userId', 'gender').dropDuplicates().count()


462

# Question 4

How many songs were played from the most played artist?

In [27]:
# TODO: write your code to answer question 4

user_log.filter(user_log.page == 'NextSong').select('Artist').groupBy('Artist').agg({'Artist':'count'}).withColumnRenamed('count(Artist)', 'Counter').sort(desc('Counter')).show(1)

+--------+-------+
|  Artist|Counter|
+--------+-------+
|Coldplay|     83|
+--------+-------+
only showing top 1 row



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [34]:
# TODO: write your code to answer question 5

function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

cusum = user_log.filter((user_log.page == 'NextSong') | (user_log.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))

cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()


+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+

